# Rebalance instances

### 1) Overall configurations 

In [1]:
import os
import os.path
import pandas as pd
from pprint import pprint

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/instance_settings_test_rebalancing.json"
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/profile_time_less_waiting_relax.json"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/profile_time_insertion.json"
#instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/no_rebalancing_fast.json"


dict_sl_status = {"MET":"Met", "UNMET":"Unmet"}
dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)



### 2) Loading the instance data

In [2]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]
request_log_folder = result_folder + "request_track/"
round_log_folder = result_folder + "round_track/"
vehicle_status_graph_folder = result_folder + "vehicle_status/"

# Folder where instances are located
instances_folder = instances_dic["instances_folder"]

# File name aggregated data
instance_name = instances_dic["instance_name"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'adjacency_matrix_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/network_info/adjacency_matrix.csv',
 'distances_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_m.csv',
 'durations_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_duration_s.csv',
 'instance_description': 'User can have their times worsened',
 'instance_name': 'INSERTION',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/profile_time_insertion/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'HE': 'rebalance_heuristic',
            'ID': 'instance_description',
            'IF': 'initial_fleet',

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [14]:
from functools import partial
def get_instance_settings(label_setting_dic, file_name):
    """Read file name and return instance settings dictionary"""

    print(f"Loading settings from {file_name}.")
    
    instance_settings = {}

    for setting in file_name.split("_"):
        
        try:
            config, value  = setting.split('-')
            label = label_setting_dic[config]
            instance_settings[label] = value
        
        except:
            label = label_setting_dic[setting]
            instance_settings[label] = True

    return instance_settings

# Example
test_label = "IN-instanceName_BA-30_ST-86400_MR-1000_IF-1000_MC-06_CD-3600_SR-S1_CS-AA_SD_VH_MO_RT_CT_UR"

# Create simplified signature using labels read
get_settings_dict_from_label = partial(get_instance_settings,  instances_dic["labels"])

get_settings_dict_from_label(test_label)

Loading settings from IN-instanceName_BA-30_ST-86400_MR-1000_IF-1000_MC-06_CD-3600_SR-S1_CS-AA_SD_VH_MO_RT_CT_UR.


{'instance_name': 'instanceName',
 'batch_duration': '30',
 'simulation_time': '86400',
 'max_requests': '1000',
 'initial_fleet': '1000',
 'max_capacity': '06',
 'contract_duration': '3600',
 'service_rate': 'S1',
 'customer_segmentation': 'AA',
 'allow_service_deterioration': True,
 'allow_vehicle_hiring': True,
 'allow_many_to_one': True,
 'reinsert_targets': True,
 'clear_target_list_every_round': True,
 'allow_urgent_relocation': True}

### Aggregate results (folder round_track)

Instance fields in `round_track` folder. Every line is a snapshot of a simulation round of 30 seconds (first column is round `timestamp`):

#### Request status per round
* `waiting`
* `finished`
* `denied`
* `n_requests`

#### Freelance vehicles per round
* `hired_vehicles`
* `deactivated_vehicles`

#### Vehicle status per round
* `active_vehicles`
* `enroute_count`
* `parked_vehicles`
* `origin_vehicles`
* `simulation.rebalancing`
* `stopped_rebalancing`
* `idle`
* `picking_up`
* `O1,O2,O3,O4`
* `V1,V2,V3,V4`
* `distance_traveled_cruising`
* `distance_traveled_loaded`
* `distance_traveled_rebalancing`
* `run_time`

#### Vehicle status per round (seats)
* `seat_count`
* `picking_up_seats`
* `rebalancing_seats`
* `empty_seats`
* `total_capacity`

#### Service quality
* `pk_delay`
* `total_delay`
* `A_pk,A_dp,A_count,A_unmet_slevels`
* `B_pk,B_dp,B_count,B_unmet_slevels`
* `C_pk,C_dp,C_count,C_unmet_slevels`

In [4]:
import pandas as pd

def get_results_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}round_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="timestamp",  parse_dates = True)

    # Number of requests
    total_requests = df["n_requests"].sum()
    serviced = df["finished"][-1]
    denied = df["denied"][-1]
    
    # Service quality
    sq_classes = ["A", "B", "C"]
    sq_settings = ["pk", "dp", "count", "unmet_slevels"]
    # e.g., ['A_pk', 'A_dp', 'A_count', 'A_unmet_slevels', 'B_pk', 'B_dp', 'B_count', 'B_unmet_slevels', 'C_pk', 'C_dp', 'C_count', 'C_unmet_slevels']
    sq_user_labels = ["{}_{}".format(sq_class, sq_setting) for sq_class in sq_classes for sq_setting in sq_settings]
    sq_user_dic = {l:df[l][-1] for l in sq_user_labels}
    
    # Distance traveled
    distance_cruising = df["distance_traveled_cruising"][-1]
    distance_loaded = df["distance_traveled_loaded"][-1]
    distance_rebalancing = df["distance_traveled_rebalancing"][-1]
    distance_total = distance_cruising + distance_loaded + distance_rebalancing

    # Pickup
    avg_pk_delay = df["pk_delay"].mean()
    avg_ride_delay = df["total_delay"].mean()

    # Separate occupancy labels (e.g., O1, O2, O3, etc.)
    occupancy_labels = [col for col in list(df) if 'O' in col]
    status_labels = [ "idle", "picking_up"] + occupancy_labels
    
    # Get fleet makeup
    fleet_makeup_labels = [col for col in list(df) if 'V' in col]
    fleet_makeup = {mk:df[mk][-1] for mk in fleet_makeup_labels}
    total_seats = {"seats_" + k:int(k[1:]) * v for k,v in fleet_makeup.items()}
    total_seats["total_seats"] = sum(total_seats.values())
    max_hired = df["denied"].max()

    # Filter occupancy columns
    df_occupancy = df[occupancy_labels]
    
    # Build fleet status
    df_status = pd.DataFrame(df_occupancy)
    df_status["picking_up"] = df["picking_up"]
    df_status["idle"] = df["idle"]
    
    # Smooth values
    df_occupancy = df_occupancy.rolling(window=24).mean()

    # Runtime
    time_ride_matching_ms = df["time_ride_matching_ms"].sum()
    time_update_fleet_status_ms = df["time_update_fleet_status_ms"].sum()
    time_vehicle_rebalancing_ms = df["time_vehicle_rebalancing_ms"].sum()
    total_runtime = time_ride_matching_ms + time_vehicle_rebalancing_ms + time_vehicle_rebalancing_ms
    
    # Dictionary of agreggate data
    dic_agreggate_data = {"serviced": "{:.2%}".format(serviced/total_requests),
                          "denied": "{:.2%}".format(denied/total_requests),
                          'max_hired': max_hired,
                          "total_requests": total_requests,
                          "avg_pk_delay": "{:.2f}".format(avg_pk_delay),
                          "avg_ride_delay": "{:.2f}".format(avg_ride_delay),
                          "total_runtime": "{:.2f}".format(total_runtime/1000/60),
                          "distance_cruising": "{:.2%}".format(distance_cruising/distance_total),
                          "distance_loaded": "{:.2%}".format(distance_loaded/distance_total),
                          "distance_rebalancing": "{:.2%}".format(distance_rebalancing/distance_total),
                          "distance_total": distance_total//1000}
    
    # All data
    dic_ag = {**dic_agreggate_data, **fleet_makeup}
    dic_ag = {**dic_ag, **total_seats}
    dic_ag = {**dic_ag, **sq_user_dic}

    return dic_ag

In [5]:
# Python graphs: https://python-graph-gallery.com/
# Visualization with matplotlib: https://www.oreilly.com/library/view/python-data-science/9781491912126/ch04.htmlimport matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pprint import pprint
from scipy.ndimage.filters import gaussian_filter1d
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
%matplotlib inline


def plot_vehicle_status_graph(instances_folder,
                              result_folder,
                              name_experiment,
                              replace = True,
                              smooth=None,
                              print_details = True,
                              fontsize = 'small',
                              fontsize_label = 'small',
                              show_tick_label_x = False,
                              label_y = (None, None, 'small'),
                              horizontal_legend = False,
                              fig_type = 'png',
                              custom_tw = (None, None),
                              day_sep_config = (None, None),
                              size_inches = (18,3),
                              nbins_y = 5,
                              x_data_format = ('1h', '%H'),
                              color_bg = "#FFE4E1",
                              context_font_scale = 1,
                              print_details_axis = (None, None),
                              input_dic = None,
                              show_week_days = False,
                              show_day_separator = True,
                              linewidth = 1.5,
                              tick_length = 3,
                              show_service_rate = "Service rate: ",
                              fontsize_label_sr = 'large',
                              msg = None):
    
    # Create directory
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    
    # Save path
    name_fig = "{}{}.{}".format(result_folder, "VS_" + name_experiment, fig_type)

    # Stop generation if already generated 
    if os.path.isfile(name_fig) and not replace:
        return
    
    print("Creating figure in '{}'".format(name_fig))
    
    # Get experiment data
    if input_dic:
        srate_dict = {'S1':'High', 'S2':'Moderate', 'S3':'Low'}
        fleet_size = int(input_dic["max_capacity"])
        v_cap = int(input_dic["max_capacity"])
        th = int(input_dic["simulation_time"])
        tw = int(input_dic["batch_duration"])
        cs = input_dic["customer_segmentation"]
        sr = input_dic.get("service_rate",  "NA") 
    
    
    # Standard data frequency
    # E.g., freq = {10min, 6h, 1d}, format = {'%H:%M', '%H'}
    x_freq, x_format = x_data_format
    
    # Get custom time window (.e.g., ('2011-02-01T00:00:00', '2011-02-06T00:00:00'))
    custom_tw_min, custom_tw_max = custom_tw
    
    
    # X series (dates)
    if not custom_tw_min:
        x_min = datetime.strptime('2011-02-01T00:00:00','%Y-%m-%dT%H:%M:%S')
    else:
        x_min = datetime.strptime(custom_tw_min,'%Y-%m-%dT%H:%M:%S')
    
    if not custom_tw_max:
        x_max = datetime.strptime(str(dic_th[th][1])[:-10],'%Y-%m-%dT%H:%M:%S')
    else:
        x_max = datetime.strptime(custom_tw_max, '%Y-%m-%dT%H:%M:%S')
        
    x_start = x_min + timedelta(seconds = tw)
    x_end = x_max + timedelta(seconds = tw)
    
    x_last_req = x_min + timedelta(seconds = th) + timedelta(seconds = tw)
    
    rangeDates = pd.date_range(start=x_start,
                               end=x_end,
                               freq=x_freq)
    
    rangeDatesLabel = [str(int(d.strftime(x_format))) for d in rangeDates]
    #rangeDatesLabel = [str(int(d.strftime(config[th]['format'])))+'h' for d in rangeDates]
    
    # Load results
    df = pd.read_csv("{}{}.csv".format(instances_folder, name_experiment),
                     index_col="timestamp",
                     parse_dates = True,
                     comment = "#")

    print("Firt record:", df.iloc[0].name, " -- Last record:", df.iloc[-1].name)
    
    
    print("TW:", x_min, x_max)
    
    # Number of requests
    total_requests = df["n_requests"].sum()
    time_ride_matching_ms = df["time_ride_matching_ms"].sum()
    time_update_fleet_status_ms = df["time_update_fleet_status_ms"].sum()
    time_vehicle_rebalancing_ms = df["time_vehicle_rebalancing_ms"].sum()
    total_runtime = time_ride_matching_ms + time_vehicle_rebalancing_ms + time_vehicle_rebalancing_ms
    serviced = df["finished"][-1]
    denied = df["denied"][-1]

    # Separate occupancy labels (e.g., O1, O2, O3, etc.)
    occupancy_labels = [col for col in list(df) if 'O' in col]
    status_labels = ["parked_vehicles", "simulation.rebalancing", "picking_up"] + occupancy_labels # ["origin_vehicles", "parked_vehicles", "simulation.rebalancing", "picking_up"] + occupancy_labels
    
    # Get fleet makeup
    fleet_makeup_labels = [col for col in list(df) if 'V' in col]
    fleet_makeup = {mk:df[mk][-1] for mk in fleet_makeup_labels}
    total_seats = {k:int(k[1:]) * v for k,v in fleet_makeup.items()}

    # Filter occupancy columns
    df_occupancy = df[occupancy_labels]
    
    # Build fleet status
    df_status = pd.DataFrame(df_occupancy)
    df_status["picking_up"] = df["picking_up"]
    df_status["simulation.rebalancing"] = df["simulation.rebalancing"]
    df_status["parked_vehicles"] = df["parked_vehicles"] + df["origin_vehicles"]
    
    # Smooth values
    df_occupancy = df_occupancy.rolling(window=24).mean()

    # Define axis
    df_filtered = df_status.loc[x_start:x_end]
    x = df_filtered.index.values
    print("Len x:", len(x))
    
    y = [df_filtered[c].tolist() for c in status_labels]
    
    # Define limits of y (before smoothing)
    ymin = 0
    ymax = max(y)

    # Smoth lines in y
    if smooth:
        y = gaussian_filter1d(y, sigma=smooth)

    # Get the unique hours within interval()
    hours = set(df_occupancy.index.hour.values)

    # Format legend
    graph_legend = ["P" + str(i) for i in range(1, len(occupancy_labels)+1)]
    #graph_legend = ["Parked", "Rebalancing", "Picking up"] + graph_legend #["Origin", "Parked", "Rebalancing", "Picking up"] + graph_legend 
    graph_legend = ["Parked", "Rebalancing", "Picking up"] + ["1 passenger", "2 passengers", "3 passengers", "4 passengers", "5 passengers", "6 passengers"]
    # Choosing palette. Source: https://seaborn.pydata.org/tutorial/color_palettes.html
    
    
    
    sns.set_context('paper')
    sns.set_style("ticks", {"xtick.major.size": 8, "axes.grid":True, 'axes.facecolor':'#d0d0dd','grid.color': '1.0'})
    sns.set_context("notebook", font_scale = context_font_scale)
    print(sns.axes_style())
    
    
    #sns.set_context("poster", font_scale = .5, rc={"grid.linewidth": 0.6})
    #sns.set_style("ticks", rc={'grid.color': 'black', 'axes.facecolor': color_bg})
    #sns.set_palette("RdBu_r", len(graph_legend))
    
    # sns.set_palette("viridis", len(graph_legend))
    #['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']
    # ['#081d58', '#253494', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#edf8b1', '#ffffd9']
    #sns.set_palette("YlGnBu_r", len(graph_legend))
    
    #YlGnBu_r = ['#081d58', '#e31a1c', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#edf8b1', '#ffffd9']
    YlGnBu_r = ['#081d58', '#e31a1c', '#225ea8', '#1d91c0', '#7fcdbb', '#c7e9b4', '#ffffd9']
    #YlGnBu_r = ['#081d58', '#e31a1c', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#edf8b1', '#ffffd9']
    
    #YlGnBu_r = ['#ffffcc','#c7e9b4','#7fcdbb','#41b6c4','#2c7fb8','#253494']
    #YlGnBu_r = ['#081d58', '#e31a1c', '#225ea8', '#41b6c4', '#7fcdbb', '#c7e9b4', '#ffffcc']
    # fresh_cut_day = ["#0B486B", "#40C0CB", "#F9F2E7", "#AEE239", "#8FBE00"]
    # fresh_cut_day_4 = ["#0B486B", "#F9F2E7", "#AEE239", "#8FBE00"]
    # pallete = adrift_in_dreams_4
    sns.set_palette(YlGnBu_r)
   
    print("Range: ", rangeDates)
    plt.xlim(min(x), rangeDates[-1])
    
     
    # Set the y limits making the maximum 5% greater
    # plt.ylim(ymin, ymax)
    
    # Pickup
    avg_pk_delay = df["pk_delay"].mean()
    avg_ride_delay = df["total_delay"].mean()
    
    # Plot
    #plt.stackplot(x, y, labels=graph_legend, linewidth=0.01, edgecolor='white')
    plt.stackplot(x, y, labels=graph_legend, linewidth=0.00)#, alpha = 0)#, edgecolor='white')
    
    # Line plot
#     for line_y, c  in zip(y, YlGnBu_r):
#         print(len(x), len(line_y))
#         plt.plot(x, line_y, color=c, linewidth=1.00)#, labels=graph_legend, linewidth=0.00)#, alpha = 0)#, edgecolor='white')
    
    # Define and format
    # https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xticks.html
    # {size in points, 'xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large'}
    ticks_x = []
    if show_tick_label_x:
        ticks_x = rangeDatesLabel
        
    l_y, t_y, fontsize = label_y

    if l_y is not None:
        plt.yticks(l_y,t_y,fontsize=fontsize)
        plt.ylim(l_y[0], l_y[-1])
        print("LIMITS:", l_y[0], l_y[-1])
    else:
        plt.yticks(fontsize=fontsize)
        plt.locator_params(axis='y', nbins = nbins_y)
        
        
    plt.xticks(rangeDates, ticks_x, fontsize=fontsize)
    
    # https://matplotlib.org/api/axis_api.html#matplotlib.axis.Axis.set_tick_params
    ax = plt.gca()
    t_width = linewidth
    ax.tick_params(width = t_width, length= tick_length)
    ax.spines['top'].set_linewidth(t_width)
    ax.spines['right'].set_linewidth(t_width)
    ax.spines['bottom'].set_linewidth(t_width)
    ax.spines['left'].set_linewidth(t_width)
    
    # Print x, y axis labels
    det_x, det_y = print_details_axis
    if det_x:
        plt.xlabel(det_x, fontweight='bold', fontsize=fontsize_label)
    if det_y:
        plt.ylabel(det_y, fontweight='bold', fontsize=fontsize_label)
    
    if print_details:
    
        # Position legend (Source: https://matplotlib.org/api/legend_api.html?highlight=legend#module-matplotlib.legend)
        legend = None
        # https://pythonspot.com/matplotlib-legend/
        if horizontal_legend:
            legend = plt.legend(loc = "upper center",
                   bbox_to_anchor=(0.5, -0.1),
                   ncol=len(graph_legend),
                   fontsize=fontsize,
                   edgecolor="white",
                   title="Vehicle status:")
        else:
            legend = plt.legend(loc = "upper left",
                   bbox_to_anchor=(1, 1),
                   ncol=1,
                   fontsize=fontsize,
                   edgecolor="white",
                   title="Vehicle status:")
        
        legend.get_title().set_fontsize(fontsize_label) #legend 'Title' fontsize
    
    # Line defining TW
    #plt.axvline(x = x_last_req, linewidth=1, color='r', linestyle='--')    
    
    
    if show_week_days:
        # First date week
        week_day = x_min + timedelta(hours = 10)

        # Loop days of the week
        diff_days = x_end - x_start
        print("DAYS:", diff_days)
        print(diff_days.total_seconds(), diff_days.total_seconds()/(24*3600))

        for i in range(0,7):

            # Print week day
            plt.text(week_day,
                 l_y[-1]+50,
                 week_day.strftime("%a"),
                 fontsize=fontsize_label,
                 bbox=dict(boxstyle='square,pad=0.0',
                           fc='none',
                           ec='none'))

            week_day = week_day + timedelta(hours = 24)

     ########################################################################
    # Inserting data labels ################################################

    # x position
    extra_info_coord = '2011-02-01 00:00:30'
    x_label = datetime.strptime(extra_info_coord, '%Y-%m-%d %H:%M:%S')
    val = [(sq_class, df_filtered.loc[extra_info_coord][sq_class]) for sq_class in status_labels]
    
    y_pos_labels = []
    previous = 0
    total = 0
    for status, v in val:
        total+=v
        y_pos_labels.append((previous, v//2 + previous, previous + v, v, status))
        previous = previous + v
    
    y_pos_labels.append((previous, v//2 + previous, previous + v, total, 'total'))
    
    bar = []
    bottom = 0
    width_bar = 0.02
    
    print("Data per status:")
    pprint(y_pos_labels)
    left = True
    color_line = 'blueviolet'
    for l, m, r, label, status in y_pos_labels:
        #bar.append(label)
        
       
        #ax.annotate(label, xy=(extra_info_coord, m-10), facecolor='red', alpha=0.5, edgecolor='red', textcoords='data', zorder= 1001)
#         t = plt.text(x_label + timedelta(minutes=60), m, label, bbox=dict(boxstyle='square,pad=0.1',
#                            alpha=0.8,
#                            fc='white',
#                            ec='none'),
#                     zorder= 1001)
        if left == True:
            x_pos_tag = x_label + timedelta(minutes=60)
            side = ((x_label + timedelta(minutes=120), m))
#             left = not left
#         else:
#             x_pos_tag = x_label - timedelta(minutes=60)
#             side = ((x_label - timedelta(minutes=120) - timedelta(minutes=240), m))
#             left = not left
        

        #print(left, side)
        print(side, (x_label, m+30))
        ax.annotate("{} {}".format(label, status), xy=(x_label, m+30),
                    xytext=side,
                    bbox=dict(
                        boxstyle='square,pad=0.1',
                        alpha=0.8,
                        fc='white',
                        ec='none'),
                    textcoords='data',
                    zorder= 1001)#,
                    #arrowprops=dict(arrowstyle='-', color='violet', linewidth=1))
        #arrowprops=dict(arrowstyle='->'), xytext=(15, -10)
        #t.set_bbox(dict(facecolor='red', alpha=0.5, edgecolor='red'))
        #plt.plot([x_label, x_label], [l,r], '-', linewidth=6, markersize=6, marker = "s", zorder= 2000)
        plt.plot([x_label, x_label], [l,r], linewidth=1, zorder=2000, markersize=6, marker = "_", color=color_line)
        
        #plt.bar([x_label], (label,), width_bar, bottom = (bottom,), linewidth=0, zorder= 1000)
        #bottom += label
        
        
    #plt.axvline(x= x_label, linewidth=1, color='black', alpha=1)
    
    
    if show_day_separator:
        # Print day separator
        back_day_sep_dic, front_day_sep_dic = day_sep_config

        # Day of the week line separator
        dashed_line = x_min

        for i in range(0,6):

            # Print day separator
            dashed_line = dashed_line + timedelta(hours = 24)
            
            #if dashed_line != x_label: 
            if back_day_sep_dic:
                back_day_sep_dic["x"] = dashed_line
                # White box below dashed line for constrast
                plt.axvline(**back_day_sep_dic)

            if front_day_sep_dic:
                front_day_sep_dic["x"] = dashed_line
                # Dashed line to separate week days
                plt.axvline(**front_day_sep_dic)
    
    # Remove white margins
    plt.margins(0,0)
    
    # PRINT SERVICE RATE
    # String format - https://docs.python.org/2/library/string.html#formatstrings
    # font - https://matplotlib.org/gallery/text_labels_and_annotations/fonts_demo_kw.html
    # size - {size in points, 'xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large'}
    
    if show_service_rate is not None:
        
        sr_pos_x = x_max
        sr_pos_y = t_y[-1]
        text = "{}{: >7.2%}".format(show_service_rate, serviced/total_requests)
        
        at = AnchoredText(text,
                 prop=dict(fontstyle='italic',
                     fontsize=fontsize_label_sr,
                     transform=ax.transAxes,
                 bbox=dict(boxstyle='square,pad=0.1',
                           alpha=0.6,
                           fc='white',
                           ec='none')), frameon=True, loc='upper right', pad=0.0, borderpad=0.3)
        
        at.patch.set_boxstyle("square,pad=0.0")
        at.patch.set_ec('none')
        at.patch.set_fc('none')
        ax.add_artist(at)
        

    if msg:
        msg_text, msg_fontsize = msg
        at = AnchoredText(msg_text,
                 prop=dict(fontstyle='italic',
                 fontsize=msg_fontsize,
                 transform=ax.transAxes,
                 bbox=dict(boxstyle='square,pad=0.0',
                           alpha=0.5,
                           fc='none',
                           ec='none')), frameon=True, loc='upper left', pad=0.0, borderpad=0.5)
        
        at.patch.set_boxstyle("square,pad=0.0")
        at.patch.set_ec('none')
        at.patch.set_fc('none')
        ax.add_artist(at)

    
    # Remove white margins
    plt.margins(0,0)
    
    fig = plt.gcf()
    fig.set_size_inches(size_inches[0], size_inches[1])
    
    #plt.savefig(name_fig, bbox_inches="tight", pad_inches=0)
    plt.savefig(name_fig, bbox_inches="tight", dpi=300)
    
    # plt.text(x_max, 1000, "  INPUT:\n    Fleet size: {}({}) \n  Service rate: {:<10} \n  Segmentation: {}  \n   #Extensions: {} \n  Rebal. after: {}m \n    Drop after: {}m \n\n  OUTPUT:\n     #Requests: {} \n  Pickup delay: {:6.2f}s\n    Ride delay: {:6.2f}s \n       Runtime: {:6.2f}m".format(fleet_size, v_cap, sr, cs , ext, rebal*tw//60, rebal*tw*deact//60, serviced, avg_pk_delay, avg_ride_delay, total_runtime/1000/60),  size="xx-small", family='monospace')
    
    # Close figure
    plt.clf()
    
    print("Fleet makeup: {}\n Total seats: {}".format(fleet_makeup, total_seats))
    print("Service rate: {:.2%} (Serviced: {} + Denied: {} = {}) \nPickup delay: {:.2f} / Ride delay: {:.2f} \nRuntime: {:.2f}s".format(serviced/total_requests, serviced, denied, total_requests, avg_pk_delay, avg_ride_delay, total_runtime/1000))
    
    # pprint(df_status)

In [15]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)


# Instance Info (key = instance name)
dic_all_instance_settings = dict()
dic_all_round_track = dict()
dic_all_request_track = dict()

print("Reading files in folder:", request_log_folder)
for file_name in instance_file_names:
    instance, extension = file_name.split(".")
    print("  - Processing", instance)
    
    
    instance_settings = get_settings_dict_from_label(instance)

    plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            instance,
                            replace=True,
                            smooth=0,
                            # label_y = ([750, 1000, 1250, 1500, 1750, 2000], [750, 1000, 1250, 1500, 1750, 2000], "xx-large"),
                            #  label_y = ([0, 250, 500, 750, 1000, 1250], [0, 250, 500, 750, 1000, 1250], "xx-large"),
                            # label_y = ([0, 25, 50], [0, 25, 50], "xx-large"),
                            label_y = ([0, 100, 200, 300, 400, 500], [0, 100, 200, 300, 400, 500], "xx-large"),
                            print_details=False,
                            context_font_scale = 1.3,
                            # size_inches = (21,3),
                            size_inches = (9, 9),
                            fontsize="xx-large",
                            fontsize_label="xx-large",
                            # custom_tw = ('2011-02-01T00:00:00', '2011-02-08T00:00:00'),
                            custom_tw = ('2011-02-01T00:00:00', '2011-02-01T23:59:59'),
                            x_data_format = ('6h', '%H'),
                            color_bg = "#C0C0C0",
                            horizontal_legend = True,
                            day_sep_config = (dict(linewidth=6, color='white', alpha=0.4), dict(linewidth=1, color='black', linestyle='--')),
                            fig_type = 'png',
                            print_details_axis = ("Time (h)", "#Vehicles"),
                            input_dic = instance_settings)

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/profile_time_insertion/request_track/
  - Processing IN-INSERTION_BA-30_ST-86400_MR-1000_IF-500_MC-4_CS-BB_RE_CT_MO_RT_UR_OP
Loading settings from IN-INSERTION_BA-30_ST-86400_MR-1000_IF-500_MC-4_CS-BB_RE_CT_MO_RT_UR_OP.
Creating figure in 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/profile_time_insertion/vehicle_status/VS_IN-INSERTION_BA-30_ST-86400_MR-1000_IF-500_MC-4_CS-BB_RE_CT_MO_RT_UR_OP.png'
Firt record: 2011-02-01 00:00:30  -- Last record: 2011-02-02 00:35:30
TW: 2011-02-01 00:00:00 2011-02-01 23:59:59
Len x: 2880
{'axes.facecolor': '#d0d0dd', 'axes.edgecolor': '.15', 'axes.grid': True, 'axes.axisbelow': True, 'axes.labelcolor': '.15', 'figure.facecolor': 'white', 'grid.color': '1.0', 'grid.linestyle': '-', 'text.color': '.15', 'xtick.color': '.15', 'ytick.color': '.15', 'xtick.direction': 'out', 'ytick.direction': 'out', 'lines.solid_capstyle': 'round', 'patch.edgecolor': 'w', 'image

<Figure size 648x648 with 0 Axes>

### Comparing hiring for scenario S2 and Segmentation BB

In [ ]:
file_name_reb_yes = "IN-WEEKDENY_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR"
file_name_reb_no = "IN-WEEKDENY_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB"

instance_settings = get_instance_settings(file_name)

general_config = dict(replace=True,
                    smooth=5,
                    print_details=False,
                    size_inches = (18,3),
                    fontsize="medium",
                    fontsize_label="large",
                    custom_tw = ('2011-02-01T00:00:00', '2011-02-08T00:00:00'),
                    x_data_format = ('3h', '%H'),
                    label_y = ([0, 125, 250, 375, 500, 625, 750, 875, 1000], [0, 125, 250, 375, 500, 625, 750, 875, 1000], "medium"),
                    color_bg = "#C0C0C0",
                    horizontal_legend = True,
                    day_sep_config = (dict(linewidth=6, color='white', alpha=0.4), dict(linewidth=1, color='black', linestyle='--')),
                    fig_type = 'svg',
                    linewidth = 1,
                    input_dic = instance_settings)
msg_fontsize = 'large'
plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            file_name_reb_no,
                            
                            print_details_axis = (None, "#Vehicles"),
                          show_week_days = True,
                            **general_config)

plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            file_name_reb_yes,
                            print_details_axis = ("Time(h)", "#Vehicles"),
                          show_tick_label_x = True,
                            **general_config)



In [ ]:
file_name_0 = "IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S2_VH_SD_RE_CT_RT_UR"
file_name_1h = "IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-3600_SR-S2_VH_SD_RE_CT_RT_UR"
file_name_3h = "IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-10800_SR-S2_VH_SD_RE_CT_RT_UR"


# presentation_config = dict(
#                     context_font_scale = 1.3,
#                     fontsize="x-large",
#                     fontsize_label = "x-large",
#                     fig_type = "png",
#                     label_y = ([0, 450, 900, 1350, 1800], [0, 450, 900, 1350, 1800], "x-large"),
#                     show_service_rate = None
#                     )

# presentation_config = dict(
#                     context_font_scale = 1.3,
#                     fontsize="x-large",
#                     fontsize_label = "x-large",
#                     fig_type = "png",
#                     label_y = ([900, 1125, 1350, 1575, 1800], [900, 1125, 1350, 1575, 1800], "x-large"),
#                     show_service_rate = None
#                     )

presentation_config = {}

general_config = dict(replace=True,
                    smooth=5,
                    print_details=False,
                    size_inches = (18,3),
                    fontsize="medium",
                    fontsize_label="medium",
                      show_service_rate = None,
                    custom_tw = ('2011-02-01T00:00:00', '2011-02-08T00:00:00'),
                    x_data_format = ('3h', '%H'),
                    label_y = ([0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800], [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800], "small"),
                    color_bg = "#C0C0C0",
                    horizontal_legend = True,
                    day_sep_config = (dict(linewidth=6, color='white', alpha=0.4), dict(linewidth=1, color='black', linestyle='--')),
                    fig_type = 'svg',
                    linewidth = 1,
                    input_dic = instance_settings)
msg_fontsize = 'large'

general_config = {**general_config, **presentation_config}

plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            file_name_0,
                          show_week_days = True,
                            print_details_axis = (None, "#Vehicles"),
                            **general_config)

plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            file_name_1h,
                            print_details_axis = (None, "#Vehicles"),
                            **general_config)

plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            file_name_3h,
                            show_tick_label_x = True,
                            print_details_axis = ("Time (h)", "#Vehicles"),
                            **general_config)

### Read all files in folder

In [ ]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)


# Instance Info (key = instance name)
dic_all_instance_settings = dict()
dic_all_round_track = dict()
dic_all_request_track = dict()

print("Reading files in folder:", request_log_folder)
cont = 0
general_config = dict(replace=True,
                    smooth=0,
                    print_details=False,
                    size_inches = (18,3),
                    fontsize="x-large",
                    fontsize_label="x-large",
                    custom_tw = ('2011-02-01T00:00:00', '2011-02-08T00:00:00'),
                    x_data_format = ('6h', '%H'),
                    label_y = ([0, 225, 450, 675, 900, 1125, 1350, 1575, 1800, 2025, 2250], [0, 225, 450, 675, 900, 1125, 1350, 1575, 1800,  2025, 2250], "small"),
                    color_bg = "#C0C0C0",
                    context_font_scale = 1.3,
                    horizontal_legend = True,
                    day_sep_config = (dict(linewidth=6, color='white', alpha=0.4), dict(linewidth=1, color='black', linestyle='--')),
                    fig_type = 'png',
                    linewidth = 1,
                    input_dic = instance_settings)
msg_fontsize = 'large'

for file_name in instance_file_names:
    #if cont >1:
    #    break
    cont = cont + 1
    instance, extension = file_name.split(".")
    print("  - Processing", instance)
   
    plot_vehicle_status_graph(round_log_folder,
                            vehicle_status_graph_folder,
                            instance,
                            show_tick_label_x = True,
                            show_week_days = True,
                            print_details_axis = ("Time (h)", "#Vehicles"),
                            **general_config)

### Create rebalance matrix

In [ ]:
  -- -- -- -- MO MO MO MO
CT                       --
CT                       --
CT                       RI
CT                       RI
--                       RI
--                       RI
--                       --
  -- -- UR UR UR UR -- --

In [ ]:
no_rebal = "VS_IN-FIXED_FLEET_NO_REBAL_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB.pdf"

list_rebal = [
    [
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT.pdf"
    ],
    [
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO_RT.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT.pdf"
    ],
    [
    
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO_RT.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_RT.pdf"
    ],
    [
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE.pdf",
    ]
]

for i in range(0, 4):
    for j in range(0, 4):
        instance, extension = list_rebal[i][j].split(".")
        print("  - Processing", instance)

        plot_vehicle_status_graph(round_log_folder,
                                vehicle_status_graph_folder,
                                instance,
                                replace=True,
                                smooth=1,
                                label_y = ([0, 250, 500, 750, 1000], ["","","","",""], "small"),
                                show_tick_label_x = False,
                                print_details=False,
                                size_inches = (2,2),
                                fontsize="medium",
                                show_week_days = False,
                                show_service_rate = "",
                                fontsize_label="medium",
                                custom_tw = ('2011-02-01T00:00:00', '2011-02-02T00:00:00'),
                                x_data_format = ('4h', '%H'),
                                color_bg = "#C0C0C0",
                                horizontal_legend = True,
                                fig_type = 'pdf',
                                linewidth = 1,
                                print_details_axis = (None, None),
                                input_dic = instance_settings)
        
